# Identifying Edible Plants
## w251 Final Project
## Scott Xu, Divya Babu, Aaron Olson

The Intent of this final project is to develop an image recognition program that can accurately identify edible and/or poisonous plants in the wild. This endeavor has been attempted by several apps and other programs - however all of these realize an edge architecture that relies on a remote server connection in order to upload the file and run through the model. 

This paper explores the difference performance options in order to arrive at the best performing model. We then work to reduce the model size in order to fit on an edge device for real time diagnosis. 

In order to get a baseline model for image recognition, we used a transfer learning technique where the model weights and architecture of ResNet50 was applied. ResNet50 was chosen for its performance as well as its size. Training on the volume of images for the duration that ResNet50 was done would not be reasonable - therefore we have used this baseline model to improve the baseline prediction. On top of this we explore different model architectures in order to define which architecture performs the best.

In order to get the best performing model we needed to remember to balance model performance with edge device performance. In the case of poisonous plants the consequences of a bad prediction can be high - however the utility of an app that takes 60 min to make a prediction is impacted. Therefore at the end of this notebook we examine the relationship with building the model on a virtual machine (for training) vs inference on the edge device (time to predict vs accuracy). 

We begin by examining the training dataset:

In [ ]:
# TODO: Insert cells with example pictures / labels and a few details about training size / etc

In [1]:
# Load ResNet50 baseline model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


### Image Augmentation

With the baseline model established, we understand that there will likely be a difference in the cleanliness of the images taken for the training dataset, vs the images taken in the field when a user wants to succesfully identify a plant. We therefore utilize image augmentation, to achieve a couple of tasks: (1) affect the image quality, orientation, etc in order to make the model more versatile (2) create more training images in order to train the model. 

Below we explore different effects of image augmentation and show below the effects of model performance: 

In [2]:
# Image augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from glob import glob

TRAIN_DIR = "train_poison"
result = [y for x in os.walk(TRAIN_DIR) for y in glob(os.path.join(x[0], '*.jpg'))]
classes = list(set([y.split("/")[-2] for y in result]))
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 32

train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 6963 images belonging to 70 classes.


In [4]:
base_model.output.shape

TensorShape([Dimension(None), Dimension(10), Dimension(10), Dimension(2048)])

### Model Architecture - Predicting Plant Class

Building on the baseline model, we have explored adding different layers (size, type, etc) in order to produce the 'best' performing model. See above and later discussions as to how we define 'best' model. For this paper we explored both accurately predicting the class of the plant in an image - which would help users understand more information about the specific plant they have taken the picture of, as well as predicting whether a plant is poisonous or not without the context of exact plant type. 

The original intent of this paper was to predict the plant class whereby the metadata associated with that prediction would yield the label of poisonous or edible - however we also chose to explore whether deep learning could accurately predict the binary choice of poisonous or not. 

In [5]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    
#     x = Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=base_model.output.shape)(x)
#     #x = Conv2D(64, (3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(dropout)(x)
#     #x.add(Flatten())
    x = Flatten()(x)
    for fc in fc_layers:
        # Can look here if adding different types of layers has an effect
        # Also explore differences in changing activation function
        # Can also iterate on droupout amount
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

# Can change the model architecture here
FC_LAYERS = [128,32]
dropout = 0.3

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(classes))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import datetime
import numpy   
    
# For the baseline model will use 100 epochs and 15000 images to test model performance
# Will then use 'optimized' model parameters to train for longer time and explore
# Size vs Accuracy for edge compute purposes
NUM_EPOCHS = 1
BATCH_SIZE = 64
num_train_images = 15000

adam = Adam(lr=0.00001)
# Can look into whether 
#finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# Checkpoin is overwritten at each epoch - can look at line below where datetime is used to create time based file names
#filepath="/root/w251_finalproject/checkpoint/model"
#checkpoint = ModelCheckpoint(filepath, monitor=["categorical_accuracy"], verbose=1, mode='max')
# log_dir = "C:\\Users\\AOlson\\Documents\\UC Berkeley MIDS\\w251_Scaling\\Final Project\\Data\\log_dir\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True)# , callbacks=[checkpoint])

numpy.savetxt('edible_loss_history.txt', numpy.array(history.history['loss']), delimiter = ',')
numpy.savetxt('edible_acc_history.txt', numpy.array(history.history['categorical_accuracy']), delimiter = ',')

Instructions for updating:
Colocations handled automatically by placer.


NameError: name 'finetune_model' is not defined

In [8]:
finetune_model

### Save Model for deployment

In [9]:
os.makedirs('./model', exist_ok=True)
finetune_model.save('./model/test.h5')

### Convert Keras model to TensorFlow PB file

In order to get the best inference performance on TX2, we utilized the TensorRT which leverage lower precision (FP16 and INT8) on GPUs. As first step, Keras model is converted to tensorflow pb file.

In [1]:
from keras import backend as K
import tensorflow as tf
from  tensorflow.python.keras.models import load_model
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

Using TensorFlow backend.


In [2]:
model = load_model('./model/test.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [7]:
model

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
#from show_graph import show_graph
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)
graph_def = sess.graph.as_graph_def()
# graph_def
#show_graph(graph_def)

In [6]:
print(model.inputs)

print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 300, 300, 3) dtype=float32>]
[<tf.Tensor 'dense_2/Softmax:0' shape=(?, 70) dtype=float32>]


In [7]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [8]:
frozen_graph = freeze_session(sess,
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 988 variables.
INFO:tensorflow:Converted 988 variables to const ops.


In [10]:
tf.train.write_graph(frozen_graph, "model", "test.pb", as_text=False)

'model/test.pb'

### Convert to checkpoint

In [6]:
saver = tf.train.Saver()

sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)
save_path = saver.save(sess, "model.ckpt")